In [3]:
from numpy import pi, ones, zeros, uint8, where, cos, sin, sqrt
from cv2 import VideoCapture, cvtColor, Canny, line, imshow, waitKey, destroyAllWindows, COLOR_BGR2GRAY, HoughLinesP
from cv2 import threshold, THRESH_BINARY, dilate, floodFill, circle, HoughLines, erode, rectangle, VideoWriter, VideoWriter_fourcc
from TraceHeader import videoFile, findIntersection, calculatePixels
from CourtMapping import courtMap, showLines, showPoint, heightP, widthP, givePoint
#from BodyTracking import bodyMap
#from mediapipe import solutions
from BallDetection import BallDetector
from BallMapping import euclideanDistance, withinCircle
import cv2
import sys
if sys.platform == "win32":
    root = 'C:/Users/Jason Wang/OneDrive - University of Virginia/'
    #os.chdir(root + 'Coding/pose_youtube/code/scripts')
else: 
    root = '/Users/jasonwang/OneDrive - University of Virginia/'

In [18]:
import cv2
import numpy as np
from mmdet.apis import DetInferencer

# Initialize the DetInferencer with the pre-trained model
inferencer = DetInferencer(model='rtmdet_tiny_8xb32-300e_coco', device='cuda:0')  # Use 'cpu' if GPU is unavailable

# Function to calculate the proportion of white pixels in the detected region
def white_clothing_ratio(image, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    person_roi = image[y1:y2, x1:x2]
    hsv = cv2.cvtColor(person_roi, cv2.COLOR_BGR2HSV)
    # Define range for white color in HSV
    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([180, 55, 255], dtype=np.uint8)
    # Create a mask for white regions
    mask = cv2.inRange(hsv, lower_white, upper_white)
    # Calculate the proportion of white pixels
    white_ratio = cv2.countNonZero(mask) / mask.size
    return white_ratio

# Load the image
image_path = 'C:/Users/Jason Wang/Downloads/Screenshot 2024-11-11 164022.png'  # Replace with your image path
image = cv2.imread(image_path)

# Perform inference to detect objects
result = inferencer(image)

# Extract bounding boxes, scores, and labels for detected objects
predictions = result['predictions'][0]
bboxes = predictions['bboxes']
scores = predictions['scores']
labels = predictions['labels']

# Assuming the 'person' class corresponds to label index 0
person_label_index = 0
white_clothing_candidates = []

# Process each detected bounding box to identify people and their white clothing ratio
for bbox, score, label in zip(bboxes, scores, labels):
    if label == person_label_index and score >= 0.5:  # Check if it’s a person with sufficient confidence
        white_ratio = white_clothing_ratio(image, bbox)
        white_clothing_candidates.append((bbox, white_ratio))

# Sort detected people by white clothing ratio and select the top two
white_clothing_candidates.sort(key=lambda x: x[1], reverse=True)
top_two_white_clothing_bboxes = [bbox for bbox, ratio in white_clothing_candidates[:5]]

# Output the bounding boxes of the top two people most likely wearing white clothing
print("Bounding boxes of top two people most likely wearing white clothing:")
for bbox in top_two_white_clothing_bboxes:
    print(bbox)  # Print coordinates [x1, y1, x2, y2]

# Optional: Draw bounding boxes on the image and save/display
for bbox in top_two_white_clothing_bboxes:
    x1, y1, x2, y2 = map(int, bbox)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Save and display the output image
# output_image_path = 'output_image.jpg'
# cv2.imwrite(output_image_path, image)
cv2.imshow('Top Two People Most Likely Wearing White Clothing', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_tiny_8xb32-300e_coco/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



Bounding boxes of top two people most likely wearing white clothing:
[720.9735717773438, 871.7401733398438, 916.0457153320312, 1144.1134033203125]
[786.3935546875, 249.7905731201172, 876.31689453125, 368.9035339355469]
[1141.053466796875, 205.12342834472656, 1243.78759765625, 430.9745788574219]
[1473.857666015625, 250.65194702148438, 1555.0589599609375, 367.5961608886719]
[623.0903930664062, 209.2816162109375, 676.822509765625, 369.05712890625]


In [20]:
from mmdet.apis import DetInferencer

# Initialize the DetInferencer with a pre-trained model
det_inferencer = DetInferencer(model='rtmdet_tiny_8xb32-300e_coco', device='cuda:0')  # Use 'cpu' if GPU is unavailable
import cv2
import numpy as np

def white_clothing_ratio(image, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    person_roi = image[y1:y2, x1:x2]
    hsv = cv2.cvtColor(person_roi, cv2.COLOR_BGR2HSV)
    # Define range for white color in HSV
    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([180, 55, 255], dtype=np.uint8)
    # Create a mask for white regions
    mask = cv2.inRange(hsv, lower_white, upper_white)
    # Calculate the proportion of white pixels
    white_ratio = cv2.countNonZero(mask) / mask.size
    return white_ratio
# Load the image
image_path = 'C:/Users/Jason Wang/Downloads/Screenshot 2024-11-11 164022.png'  # Replace with your image path
image = cv2.imread(image_path)

# Perform inference to detect objects
result = det_inferencer(image)

# Extract bounding boxes, scores, and labels for detected objects
predictions = result['predictions'][0]
bboxes = predictions['bboxes']
scores = predictions['scores']
labels = predictions['labels']

# Assuming the 'person' class corresponds to label index 0
person_label_index = 0
white_clothing_candidates = []

# Process each detected bounding box to identify people and their white clothing ratio
for bbox, score, label in zip(bboxes, scores, labels):
    if label == person_label_index and score >= 0.5:  # Check if it’s a person with sufficient confidence
        white_ratio = white_clothing_ratio(image, bbox)
        white_clothing_candidates.append((bbox, white_ratio))

# Sort detected people by white clothing ratio and select the top five
white_clothing_candidates.sort(key=lambda x: x[1], reverse=True)
top_five_white_clothing_bboxes = [bbox for bbox, ratio in white_clothing_candidates[:5]]
from mmpose.apis import MMPoseInferencer

# Initialize the MMPoseInferencer with a pre-trained model
pose_inferencer = MMPoseInferencer(pose3d='human3d', device='cuda:0')  # Use 'cpu' if GPU is unavailable
# Prepare the list of person images for pose estimation
person_images = []
for bbox in top_five_white_clothing_bboxes:
    x1, y1, x2, y2 = map(int, bbox)
    person_roi = image[y1:y2, x1:x2]
    person_images.append(person_roi)

# Perform pose estimation
pose_results = []
for person_image in person_images:
    result = pose_inferencer(person_image)
    pose_results.append(result)

# Optional: Draw bounding boxes and pose keypoints on the image
for bbox, pose_result in zip(top_five_white_clothing_bboxes, pose_results):
    x1, y1, x2, y2 = map(int, bbox)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # Draw keypoints
    for keypoint in pose_result['predictions'][0]['keypoints']:
        kp_x, kp_y = int(keypoint[0]), int(keypoint[1])
        cv2.circle(image, (kp_x, kp_y), 3, (0, 0, 255), -1)

# Save and display the output image
# output_image_path = 'output_image.jpg'
# cv2.imwrite(output_image_path, image)
cv2.imshow('Pose Estimation on People Wearing White Clothing', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


c:\Users\Jason Wang\.conda\envs\openmmlab\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_tiny_8xb32-300e_coco/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



c:\Users\Jason Wang\.conda\envs\openmmlab\lib\site-packages\mmengine\visualization\visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/body_3d_keypoint/pose_lift/h36m/motionbert_ft_h36m-d80af323_20230531.pth
11/11 17:13:26 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth
11/11 17:13:26 - mmengine - WARNING - The current default scope "mmpose" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e

TypeError: 'generator' object is not subscriptable

In [9]:
result

{'predictions': [{'labels': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    38,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    38,
    0,
    0,
    0,
    56,
    38,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    56,
    0,
    0,
    0,
    0,
    28,
    0,
    0,
    0,
    0,
    74,
    0,
    56,
    0,
    0,
    0,
    0,
    0,
    0,
    56,
    0,
    0,
    0,
    0,
    0,
    24,
    34,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    24,
    56,
    0,
    13,
    24,
    28,
    56,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    24,
    0,
    0,
    0,
    0,
    0,
    0,
    26,
    0,
    56,
    56,
    34,
    56,
    0,
    0,
    56,
    0,
    0,
    0,
    0,
    26,
    0,
    1,
    0,
    38,
    13,
    56,
    26,
    0,
    0,
 

In [9]:
video = VideoCapture(videoFile)
width = int(video.get(3))
height = int(video.get(4))

fourcc = VideoWriter_fourcc(*'mp4v')
clip = VideoWriter(root + 'Coding/pose_youtube/data/videos/broadcast_tennis/example.mp4',fourcc,25.0,(widthP,heightP))
processedFrame = None

# Ratios of the crop width, height, and offsets
# If centered is 1, program ignores offset and centers frame
class crop1:
    x: float = 50/100
    xoffset: float = 0/100
    xcenter: int = 1 
    
    y: float = 33/100
    yoffset: float = 0/100
    ycenter: int = 0
    
class crop2:
    x: float = 83/100
    xoffset: float = 0/100
    xcenter: int = 1 
    
    y: float = 60/100
    yoffset: float = 40/100
    ycenter: int = 0

# Calculations for pixels used in both crops
crop1 = calculatePixels(crop1, width, height)
crop2 = calculatePixels(crop2, width, height)
# Body smoothing, n is number of frames averaged
n = 3
counter = 0

# Player pose decleration 
mp_pose = solutions.pose

class body1:
    pose = mp_pose.Pose(model_complexity=2, min_detection_confidence=0.25, min_tracking_confidence=0.25)
    x: int
    xAvg: float = 0
    y: int
    yAvg: float = 0
    
class body2:
    pose = mp_pose.Pose(model_complexity=2, min_detection_confidence=0.25, min_tracking_confidence=0.25) 
    x: int
    xAvg: float = 0
    y: int
    yAvg: float = 0

# Setting reference frame lines
extraLen = width/3
class axis:
    top = [[-extraLen,0],[width+extraLen,0]]
    right = [[width+extraLen,0],[width+extraLen,height]]
    bottom = [[-extraLen,height],[width+extraLen,height]]
    left = [[-extraLen,0],[-extraLen,height]]

# Setting comparison points
NtopLeftP = None
NtopRightP = None
NbottomLeftP = None
NbottomRightP = None

ball_detector = BallDetector('TrackNet/Weights.pth', out_channels=2)
ballProximity = []
ball = None
lastSeen = None
handPoints = None
flag = [0,0,0,0]
coords = []
minDist1 = height*width
minDist2 = height*width
velocities = []
while video.isOpened():
    ret, frame = video.read()
    if frame is None:
        break
    
    # Apply filters that removes noise and simplifies image
    gry = cvtColor(frame, COLOR_BGR2GRAY)
    bw = threshold(gry, 156, 255, THRESH_BINARY)[1]
    canny = Canny(bw, 100, 200)
    
    # Using hough lines probablistic to find lines with most intersections
    hPLines = HoughLinesP(canny, 1, pi/180, threshold=150, minLineLength=100, maxLineGap=10)
    intersectNum = zeros((len(hPLines),2))
    i = 0
    for hPLine1 in hPLines:
        Line1x1, Line1y1, Line1x2, Line1y2 = hPLine1[0]
        Line1 = [[Line1x1,Line1y1],[Line1x2,Line1y2]]
        for hPLine2 in hPLines:
            Line2x1, Line2y1, Line2x2, Line2y2 = hPLine2[0]
            Line2 = [[Line2x1,Line2y1],[Line2x2,Line2y2]]
            if Line1 is Line2:
                continue
            if Line1x1>Line1x2:
                temp = Line1x1
                Line1x1 = Line1x2
                Line1x2 = temp
                
            if Line1y1>Line1y2:
                temp = Line1y1
                Line1y1 = Line1y2
                Line1y2 = temp
                
            intersect = findIntersection(Line1, Line2, Line1x1-200, Line1y1-200, Line1x2+200, Line1y2+200)
            if intersect is not None:
                intersectNum[i][0] += 1
        intersectNum[i][1] = i
        i += 1

    # Lines with most intersections get a fill mask command on them
    i = p = 0
    dilation = dilate(bw, ones((5, 5), uint8), iterations=1)
    nonRectArea = dilation.copy()
    intersectNum = intersectNum[(-intersectNum)[:, 0].argsort()]
    for hPLine in hPLines:
        x1,y1,x2,y2 = hPLine[0]
        # line(frame, (x1,y1), (x2,y2), (255, 255, 0), 2)
        for p in range(8):
            if (i==intersectNum[p][1]) and (intersectNum[i][0]>0):
                # line(frame, (x1,y1), (x2,y2), (0, 0, 255), 2)
                floodFill(nonRectArea, zeros((height+2, width+2), uint8), (x1, y1), 1) 
                floodFill(nonRectArea, zeros((height+2, width+2), uint8), (x2, y2), 1) 
        i+=1
    dilation[where(nonRectArea == 255)] = 0
    dilation[where(nonRectArea == 1)] = 255
    eroded = erode(dilation, ones((5, 5), uint8)) 
    cannyMain = Canny(eroded, 90, 100)
    
    # Extreme lines found every frame
    xOLeft = width + extraLen
    xORight = 0 - extraLen
    xFLeft = width + extraLen
    xFRight = 0 - extraLen
    
    yOTop = height
    yOBottom = 0
    yFTop = height
    yFBottom = 0
    
    # Finding all lines then allocate them to specified extreme variables
    hLines = HoughLines(cannyMain, 2, pi/180, 300)
    for hLine in hLines:
        for rho,theta in hLine:
            a = cos(theta)
            b = sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + width*(-b))
            y1 = int(y0 + width*(a))
            x2 = int(x0 - width*(-b))
            y2 = int(y0 - width*(a))
            
            # Furthest intersecting point at every axis calculations done here
            intersectxF = findIntersection(axis.bottom, [[x1,y1],[x2,y2]], -extraLen, 0, width+extraLen, height)
            intersectyO = findIntersection(axis.left, [[x1,y1],[x2,y2]], -extraLen, 0, width+extraLen, height)
            intersectxO = findIntersection(axis.top, [[x1,y1],[x2,y2]], -extraLen, 0, width+extraLen, height)
            intersectyF = findIntersection(axis.right, [[x1,y1],[x2,y2]], -extraLen, 0, width+extraLen, height)
            
            if (intersectxO is None) and (intersectxF is None) and (intersectyO is None) and (intersectyF is None):
                continue
            
            if intersectxO is not None:
                if intersectxO[0] < xOLeft:
                    xOLeft = intersectxO[0]
                    xOLeftLine = [[x1,y1],[x2,y2]]
                if intersectxO[0] > xORight:
                    xORight = intersectxO[0]
                    xORightLine = [[x1,y1],[x2,y2]]
            if intersectyO is not None:
                if intersectyO[1] < yOTop:
                    yOTop = intersectyO[1]
                    yOTopLine = [[x1,y1],[x2,y2]]
                if intersectyO[1] > yOBottom:
                    yOBottom = intersectyO[1]
                    yOBottomLine = [[x1,y1],[x2,y2]]
                    
            if intersectxF is not None:
                if intersectxF[0] < xFLeft:
                    xFLeft = intersectxF[0]
                    xFLeftLine = [[x1,y1],[x2,y2]]
                if intersectxF[0] > xFRight:
                    xFRight = intersectxF[0]
                    xFRightLine = [[x1,y1],[x2,y2]]
            if intersectyF is not None:
                if intersectyF[1] < yFTop:
                    yFTop = intersectyF[1]
                    yFTopLine = [[x1,y1],[x2,y2]]
                if intersectyF[1] > yFBottom:
                    yFBottom = intersectyF[1]
                    yFBottomLine = [[x1,y1],[x2,y2]]
            #line(frame, (x1,y1), (x2,y2), (0, 0, 255), 2)
    
    # lineEndpoints = []
    # lineEndpoints.append(xOLeftLine)
    # lineEndpoints.append(xORightLine)
    # lineEndpoints.append(yOTopLine)
    # lineEndpoints.append(yOBottomLine)
    # lineEndpoints.append(xFLeftLine)
    # lineEndpoints.append(xFRightLine)
    # lineEndpoints.append(yFTopLine)
    # lineEndpoints.append(yFBottomLine)
    
    # for i in range(len(lineEndpoints)):
    #     line(frame, (lineEndpoints[i][0][0],lineEndpoints[i][0][1]), (lineEndpoints[i][1][0],lineEndpoints[i][1][1]), (0, 0, 255), 2)
    
    # Top line has margin of error that effects all courtmapped outputs 
    yOTopLine[0][1] = yOTopLine[0][1]+4
    yOTopLine[1][1] = yOTopLine[1][1]+4
    
    yFTopLine[0][1] = yFTopLine[0][1]+4
    yFTopLine[1][1] = yFTopLine[1][1]+4
    
    # Find four corners of the court and display it
    topLeftP = findIntersection(xOLeftLine, yOTopLine, -extraLen, 0, width+extraLen, height)
    topRightP = findIntersection(xORightLine, yFTopLine, -extraLen, 0, width+extraLen, height)
    bottomLeftP = findIntersection(xFLeftLine, yOBottomLine, -extraLen, 0, width+extraLen, height)
    bottomRightP = findIntersection(xFRightLine, yFBottomLine, -extraLen, 0, width+extraLen, height)
    
    # If all corner points are different or something not found, rerun print
    if (not(topLeftP == NtopLeftP)) and (not(topRightP == NtopRightP)) and (not(bottomLeftP == NbottomLeftP)) and (not(bottomRightP == NbottomRightP)):
        line(frame, topLeftP, topRightP, (0, 0, 255), 2)
        line(frame, bottomLeftP, bottomRightP, (0, 0, 255), 2)
        line(frame, topLeftP, bottomLeftP, (0, 0, 255), 2)
        line(frame, topRightP, bottomRightP, (0, 0, 255), 2)
        
        circle(frame, topLeftP, radius=0, color=(255, 0, 255), thickness=10)
        circle(frame, topRightP, radius=0, color=(255, 0, 255), thickness=10)
        circle(frame, bottomLeftP, radius=0, color=(255, 0, 255), thickness=10)
        circle(frame, bottomRightP, radius=0, color=(255, 0, 255), thickness=10)
        
        NtopLeftP = topLeftP
        NtopRightP = topRightP
        NbottomLeftP = bottomLeftP
        NbottomRightP = bottomRightP
        
    else:
        line(frame, NtopLeftP, NtopRightP, (0, 0, 255), 2)
        line(frame, NbottomLeftP, NbottomRightP, (0, 0, 255), 2)
        line(frame, NtopLeftP, NbottomLeftP, (0, 0, 255), 2)
        line(frame, NtopRightP, NbottomRightP, (0, 0, 255), 2)
        
        circle(frame, NtopLeftP, radius=0, color=(255, 0, 255), thickness=10)
        circle(frame, NtopRightP, radius=0, color=(255, 0, 255), thickness=10)
        circle(frame, NbottomLeftP, radius=0, color=(255, 0, 255), thickness=10)
        circle(frame, NbottomRightP, radius=0, color=(255, 0, 255), thickness=10)

    # Displaying feet and hand points from bodyMap function
    handPointsPrev = handPoints
    feetPoints, handPoints, nosePoints = bodyMap(frame, body1.pose, body2.pose, crop1, crop2)

    if (not any(item is None for sublist in feetPoints for item in sublist)) or (not any(item is None for sublist in handPoints for item in sublist)) or (not any(item is None for sublist in nosePoints for item in sublist)):
        # circle(frame, handPoints[0], radius=0, color=(0, 0, 255), thickness=10)
        # circle(frame, handPoints[1], radius=0, color=(0, 0, 255), thickness=10)
        # circle(frame, handPoints[2], radius=0, color=(0, 0, 255), thickness=30)
        # circle(frame, handPoints[3], radius=0, color=(0, 0, 255), thickness=30)

        # circle(frame, feetPoints[0], radius=0, color=(0, 0, 255), thickness=10)
        # circle(frame, feetPoints[1], radius=0, color=(0, 0, 255), thickness=10)
        # circle(frame, feetPoints[2], radius=0, color=(0, 0, 255), thickness=30)
        # circle(frame, feetPoints[3], radius=0, color=(0, 0, 255), thickness=30)
        
        # Prioritizing lower foot y in body average y position
        if feetPoints[0][1] > feetPoints[1][1]:
            lowerFoot1 = feetPoints[0][1]
            higherFoot1 = feetPoints[1][1]
        else:
            lowerFoot1 = feetPoints[1][1]
            higherFoot1 = feetPoints[0][1]
            
        if feetPoints[2][1] > feetPoints[3][1]:
            lowerFoot2 = feetPoints[2][1]
            higherFoot2 = feetPoints[3][1]
        else:
            lowerFoot2 = feetPoints[3][1]
            higherFoot2 = feetPoints[2][1]
        
        # Allocated 75% preference to lower foot y positions
        body1.x = (feetPoints[0][0]+feetPoints[1][0])/2
        body1.y = lowerFoot1*0.8+higherFoot1*0.2

        body2.x = (feetPoints[2][0]+feetPoints[3][0])/2
        body2.y = lowerFoot2*0.8+higherFoot2*0.2
        
        # Body coordinate smoothing
        counter += 1
        coeff = 1. / min(counter, n)
        body1.xAvg = coeff * body1.x + (1. - coeff) * body1.xAvg
        body1.yAvg = coeff * body1.y + (1. - coeff) * body1.yAvg
        body2.xAvg = coeff * body2.x + (1. - coeff) * body2.xAvg
        body2.yAvg = coeff * body2.y + (1. - coeff) * body2.yAvg
        
        # Calculate euclidian distance between average of feet and hand indexes for both players
        circleRadiusBody1 = int(0.65 * euclideanDistance(nosePoints[0], [body1.x, body1.y]))
        circleRadiusBody2 = int(0.6 * euclideanDistance(nosePoints[1], [body2.x, body2.y]))
        
        # Distorting frame and outputting results
        processedFrame, M = courtMap(frame, NtopLeftP, NtopRightP, NbottomLeftP, NbottomRightP)
        # Create black background
        rectangle(processedFrame, (0,0),(967,1585),(188,145,103),2000)
        processedFrame = showLines(processedFrame)

        processedFrame = showPoint(processedFrame, M, [body1.xAvg,body1.yAvg])
        processedFrame = showPoint(processedFrame, M, [body2.xAvg,body2.yAvg])
        
        ballPrev = ball
        ball_detector.detect_ball(frame)
        if ball_detector.xy_coordinates[-1][0] is not None:
            ball = ball_detector.xy_coordinates[-1]
            lastSeen = counter
        
        # Draw a circle around both hands for both players
        # circle(frame, (handPoints[0]), circleRadiusBody1, (255,0,0), 2) # left
        circle(frame, (handPoints[1]), circleRadiusBody1, (255,0,0), 2) # right

        # circle(frame, (handPoints[2]), circleRadiusBody2, (255,0,0), 2) # left
        circle(frame, (handPoints[3]), circleRadiusBody2, (255,0,0), 2) # right
        
        if ball is not None:
            circle(frame, ball, 4, (0,255,0), 3)
            circle(frame, ballPrev, 3, (0,255,0), 2)
            
            # If ball location is unique
            if ball is not ballPrev:
                # Find locations where the ball gets closer to the body
                if withinCircle(handPoints[1], circleRadiusBody1, ball):
                    if minDist1>euclideanDistance(handPoints[1], ball):
                        minDist1 = euclideanDistance(handPoints[1], ball)
                        coords.append((ball, givePoint(M, ball), givePoint(M, (body1.x,body1.y)), counter))
                else:
                    minDist1 = circleRadiusBody1
                if withinCircle(handPoints[3], circleRadiusBody2, ball):
                    if minDist2>euclideanDistance(handPoints[3], ball):
                        minDist2 = euclideanDistance(handPoints[3], ball)
                        coords.append((ball, givePoint(M, ball), givePoint(M, (body2.x,body2.y)), counter))
                else:
                    minDist2 = circleRadiusBody2

                # Find locations of ball bounce

                if ball_detector.xy_coordinates[-2][0] is not None:
                    xVelocity = ball[0] - ballPrev[0]
                    yVelocity = (ball[1] - ballPrev[1])*(1+(height-ball[1])*0.4/height)
                    if withinCircle(handPoints[3], circleRadiusBody2, ball) or withinCircle(handPoints[1], circleRadiusBody1, ball):
                        within = True
                    else:
                        within = False
                    velocities.append(([xVelocity,yVelocity], counter, givePoint(M, ball), within))
                        

        # If the previous ball coordinate is close to the current one, remove the previous one
        if len(coords)>=2:
            if euclideanDistance(coords[-1][0], coords[-2][0]) < 200:
                del coords[-2]
        
        # Display hit points
        for i in range(len(coords)):
            circle(frame, coords[i][0], 4, (0,0,255), 4)
    
    # Write processed frame to clip
    if processedFrame is not None:    
        clip.write(processedFrame)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Last found location of ball should be appended to ball array
coords.append((ball, givePoint(M, ball), givePoint(M, ball), lastSeen))

clip.release()
video.release()
destroyAllWindows()